# HCP Vector Pipeline

## Importing Native Files

In [ ]:
import sys
sys.path.append('../Code Files')
sys.path.append('../Data')

## Import Libraries

In [ ]:
import datetime
import dill
import matplotlib.pyplot as plt
import numpy as np
from optimization import SGD, GD
import pandas as pd
import re
import scipy
from sklearn.metrics import r2_score
import torch
import seaborn as sns

## Prepare HCP Data

In [ ]:
#Load fMRI Resting State Data
with open("../Data/fmri_rs.npy", "rb") as f:
  fmri_rs = np.load(f)

#Print Shape of Data for Sanity
print(fmri_rs.shape)

#Take the Transpose so that each Sample is a Row
fmri_rs = fmri_rs.T

#Get Split to divide into train + test
mat_file = scipy.io.loadmat("../Data/MMP_HCP_60_splits.mat")
seed_1 = mat_file['folds']['seed_1']
subject_lists = seed_1[0, 0]['sub_fold'][0, 0]['subject_list']
test_subjects = [int(item[0]) for item in subject_lists[0,0].flatten()]

#Get HCP test subjects
HCP_753_Subjects = []
with open('../Data/MMP_HCP_753_subs.txt', 'r') as file:
    HCP_753_Subjects = [int(re.sub('\n', '', line)) for line in file.readlines()]

#Put the HCP test subjects into a dataframe
df = pd.read_csv("../Data/MMP_HCP_componentscores.csv")
df['Subject'] = pd.to_numeric(df['Subject'], errors='coerce')
df = df[df['Subject'].isin(HCP_753_Subjects)].reset_index(drop = True)

#Split all our data into a Train and Test Set
df_train, df_test = df[~df['Subject'].isin(test_subjects)], df[df['Subject'].isin(test_subjects)]

In [ ]:
#Create Train and Test Arrays corresponding to Training and Test Subjects
train_subjects = df_train.index.to_list()
test_subjects = df_test.index.to_list()

#Reshape Labels into Column Vectors
X_train, Y_train = fmri_rs[train_subjects], df_train["varimax_cog"].to_numpy().reshape((-1, 1))
X_test, Y_test = fmri_rs[test_subjects], df_test["varimax_cog"].to_numpy().reshape((-1, 1))

## Preprocessing Step

In [ ]:
#Row Wise Normalization of Samples
def normalize_rows(matrix: np.ndarray):
    """
    Normalize each row of the given matrix by the norm of the row.
    
    Parameters:
    matrix (numpy.ndarray): The input matrix to be normalized.
    
    Returns:
    numpy.ndarray: The row-normalized matrix.
    """
    # Calculate the L2 norm for each row. Adding a small epsilon to avoid division by zero.
    row_norms = np.linalg.norm(matrix, axis=1, keepdims=True)
    epsilon = 1e-10  # Small value to prevent division by zero
    row_norms[row_norms == 0] = epsilon
    
    # Normalize each row by its norm
    normalized_matrix = matrix / row_norms
    return normalized_matrix

#Preprocess Data
X_train = normalize_rows(X_train)
X_test = normalize_rows(X_test)

## Investigate Data

In [ ]:
# Create the histogram
sparse_counts = np.sum((X_train <= 1e-6), axis = 1)

In [ ]:
sns.histplot(sparse_counts, bins=30)

# Add titles and labels
plt.title('Histogram of Data')
plt.xlabel('Number of Values in Sample <= 1e-6')
plt.ylabel('Frequency')

# Show the plot
plt.show()

## Global Hyperparameters

In [ ]:
use_bias = False
lamb = 0.19

## Closed Form Solver

In [ ]:
from sklearn.linear_model import Ridge

#Create Ridge Model with specific alpha value and fit to (X_train, Y_train)
model = Ridge(alpha = lamb, solver = 'svd', fit_intercept = use_bias)
model.fit(X_train, Y_train)

#Get Y_train predicted using model weights
Y_train_pred = model.predict(X_train).reshape((-1, 1))
weights = model.coef_.reshape((-1, 1))
W_true = weights

#Calculate Optimal Value
p_star = (np.linalg.norm(Y_train - Y_train_pred) ** 2) + (lamb * (np.linalg.norm(weights) ** 2))
print(f"Value of p^*: {p_star}")

#Compute Training Error Metrics
train_nmse_loss = np.sum(np.square((Y_train_pred - Y_train))) / np.sum(np.square(Y_train))
train_correlation = np.corrcoef(Y_train_pred.flatten(), Y_train.flatten())[0, 1]
train_R2_score = r2_score(Y_train, Y_train_pred)
error_metrics = [train_nmse_loss, train_correlation, train_R2_score]

print('-----------------------------Training Error Metrics---------------------------------')
print(f"NMSE: {train_nmse_loss}")
print(f"Correlation: {train_correlation}")
print(f"R2_Score: {train_R2_score}")

Y_test_predicted =  model.predict(X_test).reshape((-1, 1))

#Compute Test Error Metrics
test_nmse_loss = np.sum(np.square((Y_test_predicted - Y_test))) / np.sum(np.square(Y_test))
test_correlation = np.corrcoef(Y_test_predicted.flatten(), Y_test.flatten())[0, 1]
test_R2_score = r2_score(Y_test, Y_test_predicted)
error_metrics = [test_nmse_loss,test_correlation,test_R2_score]

print('-----------------------------Testing Error Metrics---------------------------------')
print(f"NMSE: {test_nmse_loss}")
print(f"Correlation: {test_correlation}")
print(f"R2_Score: {test_R2_score}")

## Gradient Descent[Exact Line Search] Solver

In [ ]:
# max_epochs = 1000
# optimizer_code = 0

# hypers = {'lambda': lamb, 'epochs': max_epochs, 'bias': use_bias}
# W_estimated, b_estimated, losses, gap_to_optimality, nee_values, nmse_values, corr_values, R2_values = GD(X = X_train, Y = Y_train, cost_function_code = 1, hypers = hypers, p_star = p_star, W_true = W_true)

In [ ]:
# plt.plot(range(1, len(gap_to_optimality)+1), gap_to_optimality)
# plt.title('Gap to Optimality')
# plt.xlabel('Epoch')
# plt.ylabel('Gap to Optimality')
# plt.grid(True)
# plt.show()

In [ ]:
# plt.plot(range(1, len(losses)+1), losses)
# plt.title('Loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.grid(True)
# plt.show()

## Stochastic Gradient Descent Solver

Learning Rate = 1e-3:
- Noticeable difference between the Stochastic Gradients and Full Gradient

Learning Rate = 1e-4:
- Smaller difference

In [ ]:
#Cost Function to Use(0 for Least Squares, 1 for Ridge Regression)
cost_function_code = 1

#Hyperparameters
lr = 8e-4
epochs = 1000
batch_size = 256 #Set 256 as default
use_bias = False
optimizer_code = 2
decay = 0.99 #Set this as default learning rate
hypers = {'lambda': lamb, 'lr': lr, 'epochs': epochs, 'batch_size': batch_size, 'bias': use_bias, 'optimizer_code': optimizer_code, 'decay_factor': decay}

#Initial Weights
W_init = None

#Run SGD
W_estimated, b_estimated, losses, gap_to_optimality, fixed_point_stochastic_gradients, fixed_point_stochastic_gradient_norms, fixed_point_full_gradients, fixed_point_full_gradient_norms, iterates, iterate_norms, full_gradients, \
    full_gradient_norms, stochastic_gradients, stochastic_gradient_norms, nee_values, nmse_values, corr_values, R2_values, epochs_ran = SGD(X = X_train, Y = Y_train, cost_function_code = cost_function_code, hypers = hypers, optimizer_code = optimizer_code, p_star = p_star, W_true = W_true, W_init = W_init)

In [ ]:
#Fixed Point Stochastic Variance
fixed_point_stochastic_gradient_norms = fixed_point_stochastic_gradient_norms.reshape(epochs_ran, -1)
fixed_point_full_gradient_norms = fixed_point_full_gradient_norms.reshape(epochs_ran, -1)
stochastic_variance_per_epoch = np.var(fixed_point_stochastic_gradient_norms - fixed_point_full_gradient_norms, axis = 1)
plt.scatter(range(1, len(stochastic_variance_per_epoch)+1), stochastic_variance_per_epoch)
plt.title('Variance in Fixed Point Stochastic Gradient Norms')
plt.xlabel('Epoch')
plt.yscale('log')
plt.ylabel('Variance')
plt.grid(True)
plt.show()

In [ ]:
### Normalized Fixed Point Stochastic Variance
normalized_stochastic_variance_per_epoch = np.var((fixed_point_stochastic_gradient_norms - fixed_point_full_gradient_norms) / fixed_point_full_gradient_norms, axis = 1)
plt.scatter(range(1, len(normalized_stochastic_variance_per_epoch)+1), normalized_stochastic_variance_per_epoch)
plt.title('Normalized Varianced in Fixed Point Stochastic Gradient Norms')
plt.xlabel('Epoch')
plt.ylabel('Normalized Variance')
plt.grid(True)
plt.show()

In [ ]:
#Cost Function Value
plt.plot(range(1, len(losses)+1), losses)
plt.title('Cost Function Values(i.e. Loss)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.yscale('log')
plt.grid(True)
plt.show()

In [ ]:
#Gap to Closed Form Solution
plt.plot(range(1, len(gap_to_optimality)+1), gap_to_optimality)
plt.title('Gap to Closed Form Solution')
plt.xlabel('Epoch')
plt.ylabel('Gap')
plt.yscale('log')
plt.grid(True)
plt.show()

In [ ]:
#Plot Full Gradient Norms
plt.plot(range(1, len(full_gradient_norms)+1), full_gradient_norms)
plt.title('Full Gradient Norms')
plt.xlabel('Epoch')
plt.ylabel('Norm')
plt.yscale('log')
plt.grid(True)
plt.show()

### Stochastic Gradient Descent vs Full Gradients

#### Stochastic Gradients vs Full Gradients

In [ ]:
## Least Squares Term: ||XW - Y||_2^2 
## Gradient of Least Squares Term: Summation of Gradients of (<xi, w>  - yi)^2

## L2 Loss Term: ||W||_2^2 
## Gradient of L2 Loss Term: 2w

num_batches = int(len(stochastic_gradients) / epochs_ran) #number of gradients within a stochastic group

#Store Stuff
A = []
A_norms = []
A_avg = []

differences = []

#Iterate through Full Gradients
for idx in range(len(full_gradients)):
    full_gradient = full_gradients[idx] #Get Full Gradient
    
    #Get Stochastic Gradients
    start = idx * num_batches
    end = (idx + 1) * num_batches
    stochastic_slice = stochastic_gradients[start:end]
    
    #Compute Sum of Stochastic Gradients
    stochastic_sum = 0
    for idx2 in range(len(stochastic_slice)):
        stochastic_sum += stochastic_slice[idx2]
    
    expected_stochastic_gradient = stochastic_sum / num_batches #expected value of stochastic gradient. Ideally, we want this expected value to equal to the full gradient
    
    #Compute Normalized Difference
    diff = expected_stochastic_gradient - full_gradient
    norm_diff = torch.norm(diff).item()
    differences.append(norm_diff)
    
    
    normalized_diff = stochastic_sum - full_gradient
    A.append(normalized_diff)    
    A_norms.append(torch.norm(normalized_diff).item())
    A_avg.append(torch.mean(normalized_diff))

In [ ]:
plt.plot(range(1, len(differences)+1), differences)
plt.title('Difference between Expected Value of Stochastic Gradient and Full Gradient')
plt.xlabel('Epoch')
plt.ylabel('Difference[Norm]')
plt.yscale('log')
plt.grid(True)
plt.show()

In [ ]:
plt.plot(range(1, len(A_norms)+1), A_norms)
plt.title('Difference between Sum of Stochastic Gradients and Full Gradient')
plt.xlabel('Epoch')
plt.ylabel('Difference [Norm]')
plt.yscale('log')
plt.grid(True)
plt.show()

In [ ]:
plt.plot(range(1, len(A_avg)+1), A_avg)
plt.title('Average Value[Differential]')
plt.xlabel('Epoch')
plt.ylabel('Average')
plt.yscale('log')
plt.grid(True)
plt.show()

#### Fixed Point Stochastic Gradients vs Fixed Point Full Gradients

In [ ]:
## Least Squares Term: ||XW - Y||_2^2 
## Gradient of Least Squares Term: Summation of Gradients of (<xi, w>  - yi)^2

## L2 Loss Term: ||W||_2^2 
## Gradient of L2 Loss Term: 2w

num_batches = int(len(fixed_point_stochastic_gradients) / epochs_ran) #number of gradients within a stochastic group

#Store Stuff
A = []
A_norms = []
A_avg = []

differences = []

#Iterate through Full Gradients
for idx in range(len(fixed_point_full_gradients)):
    fixed_point_full_gradient = fixed_point_full_gradients[idx] #Get Full Gradient
    
    #Get Stochastic Gradients
    start = idx * num_batches
    end = (idx + 1) * num_batches
    fixed_point_stochastic_slice = fixed_point_stochastic_gradients[start:end]
    
    #Compute Sum of Stochastic Gradients
    fixed_point_stochastic_sum = 0
    for idx2 in range(len(fixed_point_stochastic_slice)):
        fixed_point_stochastic_sum += fixed_point_stochastic_slice[idx2]
    
    expected_fixed_point_stochastic_gradient = fixed_point_stochastic_sum / num_batches #expected value of stochastic gradient. Ideally, we want this expected value to equal to the full gradient
    
    #Compute Normalized Difference
    diff = expected_fixed_point_stochastic_gradient - fixed_point_full_gradient
    norm_diff = torch.norm(diff).item()
    differences.append(norm_diff)
    
    
    normalized_diff = fixed_point_stochastic_sum - fixed_point_full_gradient
    A.append(normalized_diff)    
    A_norms.append(torch.norm(normalized_diff).item())
    A_avg.append(torch.mean(normalized_diff))

In [ ]:
plt.plot(range(1, len(differences)+1), differences)
plt.title('Difference between Expected Value of Fixed Point Stochastic Gradient and Fixed Point Full Gradient')
plt.xlabel('Epoch')
plt.ylabel('Difference[Norm]')
plt.yscale('log')
plt.grid(True)
plt.show()

In [ ]:
plt.plot(range(1, len(A_norms)+1), A_norms)
plt.title('Difference between Sum of Fixed Point Stochastic Gradients and Fixed Point Full Gradient')
plt.xlabel('Epoch')
plt.ylabel('Difference [Norm]')
plt.yscale('log')
plt.grid(True)
plt.show()

In [ ]:
plt.plot(range(1, len(A_avg)+1), A_avg)
plt.title('Average Value[Differential]')
plt.xlabel('Epoch')
plt.ylabel('Average')
plt.yscale('log')
plt.grid(True)
plt.show()

In [ ]:
## Compute the Sum of the Gradients of the L2 Norms during each epoch(i.e. across all stochastic updates within the epoch)
iterate_norm_tensor_sums = []
for epoch in range(epochs_ran):
    start = epoch * (num_batches)
    end = (epoch + 1) * (num_batches)
    iterate_slice = iterates[start: end]
    
    iterate_sum = 0
    
    for idx2 in range(len(iterate_slice)):
        iterate_sum += iterate_slice[idx2]
    
    iterate_norm_tensor_sums.append(torch.norm(iterate_sum))

iterate_norm_tensor_sums

In [ ]:
plt.plot(range(1, len(iterate_norm_tensor_sums)+1), iterate_norm_tensor_sums)
plt.title('Iterate Sum Norms')
plt.xlabel('Epoch')
plt.yscale('log')
plt.grid(True)
plt.show()

In [ ]:
#Testing
weights = W_estimated
use_bias = b_estimated
Y_test_predicted = X_test @ W_estimated + use_bias

#Compute NEE, NMSE, Correlation, and R^2 Score
test_normalized_estimation_error = ((np.linalg.norm(W_estimated - W_true)) ** 2) /  ((np.linalg.norm(W_true)) ** 2)
test_nmse_loss = np.sum(np.square((Y_test_predicted - Y_test))) / np.sum(np.square(Y_test))
test_correlation = np.corrcoef(Y_test_predicted.flatten(), Y_test.flatten())[0, 1]
test_R2_score = r2_score(Y_test, Y_test_predicted)

error_metrics = [test_normalized_estimation_error,test_nmse_loss,test_correlation,test_R2_score]

print('-----------------------------Testing Error Metrics---------------------------------')
print(f"NEE: {test_normalized_estimation_error}")
print(f"NMSE: {test_nmse_loss}")
print(f"Correlation: {test_correlation}")
print(f"R2_Score: {test_R2_score}")
print(f"Bias: {use_bias}")

# Plotting both arrays
plt.figure(figsize=(10, 5))  # Set the figure size
plt.plot(Y_test_predicted, label='Predicted', color='blue', marker='o')  # Plot Y_test_predicted
plt.plot(Y_test, label='Actual', color='red', linestyle='--', marker='o')  # Plot Y_test with dashed line
plt.title(f"Comparison of Predicted and Actual Values")  # Title of the plot
plt.xlabel('Index')  # Label for the x-axis
plt.ylabel('Values')  # Label for the y-axis
plt.legend()  # Add a legend
plt.grid(True)  # Add gridlines for better readability
plt.show()  # Display the plot

#Get current time and store in variable
formatted_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
pkl_file = f"../Experimental Results/"

for param in hypers:
   pkl_file += f"{param}={hypers[param]}."

pkl_file += ".pkl"

with open(pkl_file, "wb") as file:
   dill.dump((W_estimated, b_estimated, losses, gap_to_optimality, fixed_point_stochastic_gradients, fixed_point_stochastic_gradient_norms, fixed_point_full_gradients, fixed_point_full_gradient_norms, iterates, iterate_norms, full_gradients, \
    full_gradient_norms, stochastic_gradients, stochastic_gradient_norms, nee_values, nmse_values, corr_values, R2_values, epochs_ran), file)

## Grid Search for Learning Rate

In [ ]:
lrs = np.arange(1e-5, 1e-2 + 1e-5, 1e-5)

for lr in lrs:
    #Cost Function to Use(0 for Least Squares, 1 for Ridge Regression)
    cost_function_code = 1

    #Hyperparameters
    epochs = 1000
    batch_size = 256 #Set 256 as default
    use_bias = False
    optimizer_code = 2
    decay = 0.99 #Set this as default learning rate
    hypers = {'lambda': lamb, 'lr': lr, 'epochs': epochs, 'batch_size': batch_size, 'bias': use_bias, 'optimizer_code': optimizer_code, 'decay_factor': decay}

    #Initial Weights
    W_init = None

    #Run SGD
    print(f"------------ Learning Rate = {lr} ------------")
    W_estimated, b_estimated, losses, gap_to_optimality, fixed_point_stochastic_gradients, fixed_point_stochastic_gradient_norms, fixed_point_full_gradients, fixed_point_full_gradient_norms, iterates, iterate_norms, full_gradients, \
        full_gradient_norms, stochastic_gradients, stochastic_gradient_norms, nee_values, nmse_values, corr_values, R2_values, epochs_ran = SGD(X = X_train, Y = Y_train, cost_function_code = cost_function_code, hypers = hypers, optimizer_code = optimizer_code, p_star = p_star, W_true = W_true, W_init = W_init)
        
    #Get current time and store in variable
    formatted_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    pkl_file = f"../Experimental Results/"

    for param in hypers:
        pkl_file += f"{param}={hypers[param]}."

    pkl_file += ".pkl"

    with open(pkl_file, "wb") as file:
        dill.dump((W_estimated, b_estimated, losses, gap_to_optimality, fixed_point_stochastic_gradients, fixed_point_stochastic_gradient_norms, fixed_point_full_gradients, fixed_point_full_gradient_norms, iterates, iterate_norms, full_gradients, \
        full_gradient_norms, stochastic_gradients, stochastic_gradient_norms, nee_values, nmse_values, corr_values, R2_values, epochs_ran), file)